# FEM Simulatons of Stationary Stokes Flow in Cylindrical Cavity with Rotating Lid 

Simulates [Stokes flow](https://en.wikipedia.org/wiki/Stokes_flow) for a Newtonian fluid in a shallow cylindrical cavity with rotating lid using the [Ferrite.jl](https://ferrite-fem.github.io/Ferrite.jl/stable/) finite element solver. Code adapted from the tutorial [Ferrite.jl Stokes Flow Tutorial](https://ferrite-fem.github.io/Ferrite.jl/stable/tutorials/stokes-flow/). 

<b>Problem to Solve</b>

See [Stokes flow](https://en.wikipedia.org/wiki/Stokes_flow). 

<b>Boundary Conditions</b>

At top lid: Dirichlet boundary condition for velocity. At vertical walls and bottom lid: [no-slip](https://en.wikipedia.org/wiki/No-slip_condition) boundary condition for the velocity. Average boundary condition for the pressure. 

<b>Weak or Variational Formulation</b>

See e.g. [weak formulation](https://en.wikipedia.org/wiki/Weak_formulation) for the Poisson equation. Requires extension for the Stokes equations. See e.g. Section 6.5 of book of Donea and Huerta. 

<b>Finite Element Discretization</b>

See e.g. Section 6.5 of book of Donea and Huerta.

<b>Coupled Pressure-Velocity Linear System</b>

See e.g. tutorial [Ferrite.jl Stokes Flow Tutorial](https://ferrite-fem.github.io/Ferrite.jl/stable/tutorials/stokes-flow/) and Section 6.5 of book of Donea and Huerta.

<b>Questions</b>
1. Can second order shape functions in 3D be used?
3. What does the mean pressure constraint actually do? 
4. Do pressure oscillations at the boundary of the lid disappear when using a finer mesh?
5. Do pressure oscillations at the boundary of the lid disappear when making the cavity deeper? 

<b>To do</b> 
1. repeat for various viscosity values; 
2. add post-processing for the volume-averaged strain rate. Similar to function compute_stress() in the tutorial [computational_homogenization](https://ferrite-fem.github.io/Ferrite.jl/stable/tutorials/computational_homogenization);
3. extend to non-Newtonian fluid with shear thinning. Similar to tutorial [hyperelasticity](https://ferrite-fem.github.io/Ferrite.jl/stable/tutorials/hyperelasticity);
4. try third order velocity and second order pressure; 
5. extend to Oseen model by using Stokes model as initial guess; 
6. extend to various measurement protocols;   

##  Import Packages

In [21]:
using BlockArrays
using LinearAlgebra
using UnPack
using LinearSolve 
using SparseArrays
using Ferrite
using FerriteGmsh 
using OrdinaryDiffEq
using DifferentialEquations
using Plots 
using WriteVTK

## Section 1: Introduction 

Simulates Stokes flow in a cylindrical cavity. Code copied from [Ferrite.jl Stokes Flow Tutorial](https://ferrite-fem.github.io/Ferrite.jl/stable/tutorials/stokes-flow/). Requires more details.

## Section 2: Read 2D Mesh From External Input Mesh File  

In [22]:
# grid = togrid("mesh_cavity_coarse.msh")
grid = togrid("mesh_cylinder.msh")

Info    : Reading 'mesh_cylinder.msh'...
Info    : 9 entities
Info    : 17208 nodes
Info    : 97679 elements
Info    : Done reading 'mesh_cylinder.msh'


Grid{3, Tetrahedron, Float64} with 81637 Tetrahedron cells and 17208 nodes

In [23]:
#getfacetset(grid,"wall")

In [24]:
#getcellset(grid,"volume")

## Section 3: Functions for Assembly of Stiffness Matrix and Load Vector 
Modified tutorial by removing the volumetric source term. 

In [25]:
#getfacetset(grid,"lid")

In [26]:
function lid_rotation(X, t)
    # velocity in cylindrical coordinates (0, om, 0)
    x, y, z = X
    r = sqrt(x^2+y^2)
    th = atan(y,x) 
    vmax = 20. 
    return Vec{3}((-r*vmax*sin(th), r*vmax*cos(th), 0.0)) 
end

function assemble_system!(K, f, dh, cvu, cvp,viscosity)
    assembler = start_assemble(K, f)
    ke = zeros(ndofs_per_cell(dh), ndofs_per_cell(dh))
    fe = zeros(ndofs_per_cell(dh))
    range_u = dof_range(dh, :u)
    ndofs_u = length(range_u)
    range_p = dof_range(dh, :p)
    ndofs_p = length(range_p)
    ϕᵤ = Vector{Vec{3,Float64}}(undef, ndofs_u)
    ∇ϕᵤ = Vector{Tensor{2,3,Float64,9}}(undef, ndofs_u) # 3-by-3 tensor 
    divϕᵤ = Vector{Float64}(undef, ndofs_u)
    ϕₚ = Vector{Float64}(undef, ndofs_p)
    for cell in CellIterator(dh)
        Ferrite.reinit!(cvu, cell)
        Ferrite.reinit!(cvp, cell)
        ke .= 0
        fe .= 0
        for qp in 1:getnquadpoints(cvu)
            dΩ = getdetJdV(cvu, qp)
            for i in 1:ndofs_u
                ϕᵤ[i] = shape_value(cvu, qp, i)
                ∇ϕᵤ[i] = shape_gradient(cvu, qp, i)
                divϕᵤ[i] = shape_divergence(cvu, qp, i)
            end
            for i in 1:ndofs_p
                ϕₚ[i] = shape_value(cvp, qp, i)
            end
            # u-u
            for (i, I) in pairs(range_u), (j, J) in pairs(range_u)
                ke[I, J] += viscosity*( ∇ϕᵤ[i] ⊡ ∇ϕᵤ[j] ) * dΩ
            end
            # u-p
            for (i, I) in pairs(range_u), (j, J) in pairs(range_p)
                ke[I, J] += ( -divϕᵤ[i] * ϕₚ[j] ) * dΩ
            end
            # p-u
            for (i, I) in pairs(range_p), (j, J) in pairs(range_u)
                ke[I, J] += ( -divϕᵤ[j] * ϕₚ[i] ) * dΩ
            end
            # rhs
            for (i, I) in pairs(range_u)
                x = spatial_coordinate(cvu, qp, getcoordinates(cell))
                # b = exp(-100 * norm(x - Vec{2}((0.75, 0.1)))^2)
                bv = Vec{3}((0.0, 0.0,0.0))
                fe[I] += (ϕᵤ[i] ⋅ bv) * dΩ
            end
        end
        assemble!(assembler, celldofs(cell), ke, fe)
    end
    return K, f
end

function setup_mean_constraint(dh, fvp)
    assembler = Ferrite.COOAssembler()
    # All external boundaries
    set = union(
        getfacetset(dh.grid, "lid"),
        getfacetset(dh.grid, "wall"),
    )
    # Allocate buffers
    range_p = dof_range(dh, :p)
    element_dofs = zeros(Int, ndofs_per_cell(dh))
    element_dofs_p = view(element_dofs, range_p)
    element_coords = zeros(Vec{3}, 4) # assuming mesh with tetrahedra only 
    Ce = zeros(1, length(range_p)) # Local constraint matrix (only 1 row)
    # Loop over all the boundaries
    for (ci, fi) in set
        Ce .= 0
        getcoordinates!(element_coords, dh.grid, ci)
        Ferrite.reinit!(fvp, element_coords, fi)
        celldofs!(element_dofs, dh, ci)
        for qp in 1:getnquadpoints(fvp)
            dΓ = getdetJdV(fvp, qp)
            for i in 1:getnbasefunctions(fvp)
                Ce[1, i] += shape_value(fvp, qp, i) * dΓ
            end
        end
        # Assemble to row 1
        assemble!(assembler, [1], element_dofs_p, Ce)
    end
    C, _ = finish_assemble(assembler)
    # Create an AffineConstraint from the C-matrix
    _, J, V = findnz(C)
    _, constrained_dof_idx = findmax(abs2, V)
    constrained_dof = J[constrained_dof_idx]
    V ./= V[constrained_dof_idx]
    mean_value_constraint = AffineConstraint(
        constrained_dof,
        Pair{Int,Float64}[J[i] => -V[i] for i in 1:length(J) if J[i] != constrained_dof],
        0.0,
    )

    return mean_value_constraint
end

setup_mean_constraint (generic function with 1 method)

## Section 4: Set-up, Assembly and Solve

In [32]:
dim = 3 
degree = 1

# Interpolations
ipu = Lagrange{RefTetrahedron,degree+1}() ^ 3 # quadratic for 3 velocity components 
ipp = Lagrange{RefTetrahedron,degree}()       # linear for scalar pressure 

# Dofs
dh = DofHandler(grid)
add!(dh, :u, ipu)
add!(dh, :p, ipp)
close!(dh)

# FE values
qr = QuadratureRule{RefTetrahedron}(2)
ipg = Lagrange{RefTetrahedron,1}() # linear geometric interpolation
cvu = CellValues(qr, ipu, ipg) # observe three arguments - need to document whether this is required 
cvp = CellValues(qr, ipp, ipg) # observe three arguments - need to document whether this is required
qr_facet = FacetQuadratureRule{RefTetrahedron}(2)
fvp = FacetValues(qr_facet, ipp, ipg) # required for pressure constraint 

# Boundary conditions 
ch = ConstraintHandler(dh)

# Boundary conditions part (1/3): Dirichlet BC for the velocity at the top lid 
lid = getfacetset(dh.grid, "lid")
dbc1 = Dirichlet(:u, lid, (x, t) -> lid_rotation(x,t) )
add!(ch, dbc1)

# Boundary conditions part (2/3): no slip boundary condition - impose velocity to be zero vector on the walls   
wall = getfacetset(dh.grid, "wall")
dbc2 = Dirichlet(:u, wall, (x, t) -> [0, 0, 0])
add!(ch, dbc2)
    
# Boundary conditions part (3/3): apply pressure constraint
# mean_value_constraint = setup_mean_constraint(dh, fvp)
# add!(ch, mean_value_constraint)
dbc3 = Dirichlet(:p, lid, (x, t) -> 0)
add!(ch, dbc3)
    
# Finalize
close!(ch)

# Global tangent matrix and rhs
coupling = [true true; true false] # no coupling between pressure test/trial functions
K = allocate_matrix(dh, ch; coupling=coupling)
f = zeros(ndofs(dh))

# Assemble system
viscosity = 1e3
assemble_system!(K, f, dh, cvu, cvp, viscosity); 

In [33]:
# Apply boundary conditions and solve
apply!(K, f, ch)
u = K \ f;

## Section 5: Post-Processing

In [34]:
VTKGridFile("stokes_cylinder", dh) do vtk
    write_solution(vtk, dh, u)
    Ferrite.write_constraints(vtk, ch)
end

VTKGridFile for the closed file "stokes_cylinder.vtu".